# Exploritary Data Analysis

In [2]:
import pandas as pd
train = pd.read_csv('train_house_prices.csv')
test = pd.read_csv('test_house_prices.csv')

## 1. Basics

* Number of rows
* Number of columns

In [3]:
# shape(rows, columns)
print(f'Shape of train data = {train.shape}')
print(f'Shape of test data = {test.shape}')

Shape of train data = (1460, 81)
Shape of test data = (1459, 80)


* Make a table of the data types, amount of null values and description of all the features

In [5]:
data_types = train.dtypes
null_values = train.isnull().sum()
feature_names = train.columns
new_table = pd.concat([data_types, null_values], axis=1)

In [6]:
print(new_table.to_markdown())

|               | 0       |    1 |
|:--------------|:--------|-----:|
| Id            | int64   |    0 |
| MSSubClass    | int64   |    0 |
| MSZoning      | object  |    0 |
| LotFrontage   | float64 |  259 |
| LotArea       | int64   |    0 |
| Street        | object  |    0 |
| Alley         | object  | 1369 |
| LotShape      | object  |    0 |
| LandContour   | object  |    0 |
| Utilities     | object  |    0 |
| LotConfig     | object  |    0 |
| LandSlope     | object  |    0 |
| Neighborhood  | object  |    0 |
| Condition1    | object  |    0 |
| Condition2    | object  |    0 |
| BldgType      | object  |    0 |
| HouseStyle    | object  |    0 |
| OverallQual   | int64   |    0 |
| OverallCond   | int64   |    0 |
| YearBuilt     | int64   |    0 |
| YearRemodAdd  | int64   |    0 |
| RoofStyle     | object  |    0 |
| RoofMatl      | object  |    0 |
| Exterior1st   | object  |    0 |
| Exterior2nd   | object  |    0 |
| MasVnrType    | object  |    8 |
| MasVnrArea    | fl

| Feature | DataType | Null | Description
|:--------------|:--------|-----:| -----------:|
| Id            | int64   |    0 |
| MSSubClass    | int64   |    0 |
| MSZoning      | object  |    0 |
| LotFrontage   | float64 |  259 |
| LotArea       | int64   |    0 |
| Street        | object  |    0 |
| Alley         | object  | 1369 |
| LotShape      | object  |    0 |
| LandContour   | object  |    0 |
| Utilities     | object  |    0 |
| LotConfig     | object  |    0 |
| LandSlope     | object  |    0 |
| Neighborhood  | object  |    0 |
| Condition1    | object  |    0 |
| Condition2    | object  |    0 |
| BldgType      | object  |    0 |
| HouseStyle    | object  |    0 |
| OverallQual   | int64   |    0 |
| OverallCond   | int64   |    0 |
| YearBuilt     | int64   |    0 |
| YearRemodAdd  | int64   |    0 |
| RoofStyle     | object  |    0 |
| RoofMatl      | object  |    0 |
| Exterior1st   | object  |    0 |
| Exterior2nd   | object  |    0 |
| MasVnrType    | object  |    8 |
| MasVnrArea    | float64 |    8 |
| ExterQual     | object  |    0 |
| ExterCond     | object  |    0 |
| Foundation    | object  |    0 |
| BsmtQual      | object  |   37 |
| BsmtCond      | object  |   37 |
| BsmtExposure  | object  |   38 |
| BsmtFinType1  | object  |   37 |
| BsmtFinSF1    | int64   |    0 |
| BsmtFinType2  | object  |   38 |
| BsmtFinSF2    | int64   |    0 |
| BsmtUnfSF     | int64   |    0 |
| TotalBsmtSF   | int64   |    0 |
| Heating       | object  |    0 |
| HeatingQC     | object  |    0 |
| CentralAir    | object  |    0 |
| Electrical    | object  |    1 |
| 1stFlrSF      | int64   |    0 |
| 2ndFlrSF      | int64   |    0 |
| LowQualFinSF  | int64   |    0 |
| GrLivArea     | int64   |    0 |
| BsmtFullBath  | int64   |    0 |
| BsmtHalfBath  | int64   |    0 |
| FullBath      | int64   |    0 |
| HalfBath      | int64   |    0 |
| BedroomAbvGr  | int64   |    0 |
| KitchenAbvGr  | int64   |    0 |
| KitchenQual   | object  |    0 |
| TotRmsAbvGrd  | int64   |    0 |
| Functional    | object  |    0 |
| Fireplaces    | int64   |    0 |
| FireplaceQu   | object  |  690 |
| GarageType    | object  |   81 |
| GarageYrBlt   | float64 |   81 |
| GarageFinish  | object  |   81 |
| GarageCars    | int64   |    0 |
| GarageArea    | int64   |    0 |
| GarageQual    | object  |   81 |
| GarageCond    | object  |   81 |
| PavedDrive    | object  |    0 |
| WoodDeckSF    | int64   |    0 |
| OpenPorchSF   | int64   |    0 |
| EnclosedPorch | int64   |    0 |
| 3SsnPorch     | int64   |    0 |
| ScreenPorch   | int64   |    0 |
| PoolArea      | int64   |    0 |
| PoolQC        | object  | 1453 |
| Fence         | object  | 1179 |
| MiscFeature   | object  | 1406 |
| MiscVal       | int64   |    0 |
| MoSold        | int64   |    0 |
| YrSold        | int64   |    0 |
| SaleType      | object  |    0 |
| SaleCondition | object  |    0 |
| SalePrice     | int64   |    0 |
